In [9]:
solver_grammar = """
    start: _NL? problem
    problem: ([expr | decl] _NL)+
    decl: symbol "=" expr
    expr: binoperation
        | method
        | symbol
        | literal
    method: symbol "(" parameter ")"
    parameter: expr ("," expr)*
    binoperation: expr OPERATOR expr
    symbol: SYMBOL
    literal: LITERAL
    
    SYMBOL: /[a-zA-Z][a-zA-Z0-9]*/
    LITERAL: /\d*\.?\d+/
    OPERATOR: /[\/*\-+]/
    
    %import common.INT -> NUMBER
    %import common.NEWLINE -> _NL
    %import common.WS_INLINE
    
    %ignore WS_INLINE
"""

In [10]:
from lark import Lark
parser = Lark(solver_grammar)
sample = """
t = triangle(a, b, c)
perimeter(t, 5)
eq(dist(a, b), 2)
eq(dist(b, c), 1)
solve(dist(a, c))
"""
print(parser.parse(sample).pretty())
print(parser.parse(sample))
str(parser.parse(sample).children[0].children[0].children[0].children[0])

start
  problem
    decl
      symbol	t
      expr
        method
          symbol	triangle
          parameter
            expr
              symbol	a
            expr
              symbol	b
            expr
              symbol	c
    expr
      method
        symbol	perimeter
        parameter
          expr
            symbol	t
          expr
            literal	5
    expr
      method
        symbol	eq
        parameter
          expr
            method
              symbol	dist
              parameter
                expr
                  symbol	a
                expr
                  symbol	b
          expr
            literal	2
    expr
      method
        symbol	eq
        parameter
          expr
            method
              symbol	dist
              parameter
                expr
                  symbol	b
                expr
                  symbol	c
          expr
            literal	1
    expr
      method
        symbol	solve
        parameter
          expr
    

't'

In [11]:
from problem import *
from lark import Tree, Token

p = Problem() 

methods = {
    'triangle': p.triangle,
    'perimeter': p.perimeter,
    'eq': p.eq,
    'dist': p.dist,
    'solve': p.solve
}

def process_expression(root, problem):
    if root.data == 'literal' or root.data == 'symbol':
        return(str(root.children[0]))
    elif root.data == 'binoperation':
        return(visit_branch(root, problem))
    elif root.data == 'method':
        method = visit_branch(root, problem)
        #
        return(methods.get(method[0], lambda: 'Invalid')(*method[1]))

    #return visit_branch(root)

def visit_branch(root, problem):
    if isinstance(root, Tree):
        params = [visit_branch(child, problem) for child in root.children]
       # print(params)
        if root.data == 'expr':
            return process_expression(root.children[0], problem)
        elif root.data == 'decl':
            problem.bind(*params)
        elif root.data == 'literal' or root.data == 'symbol':
            return params[0]
        return params
    if isinstance(root, Token):
        return str(root)
               
visit_branch(parser.parse(sample), p)

[[['t', <problem.Triangle at 0x2b0d7c79c18>], None, None, None, 2.0]]